# **Preparação dos dados**

In [ ]:
!pip install pyopenms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 MB 11.2 MB/s eta 0:00:00


In [ ]:
import shutil
import os

# Verifica se a pasta existe e remove ela completamente
if os.path.exists('/content/drive'):
    shutil.rmtree('/content/drive')
    print("Pasta /content/drive removida com sucesso.")

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pyopenms as po
import os

caminho_do_arquivo = '/content/drive/MyDrive/Trabalho_RNAP/dados_mzML/PG_1.mzML'


if not os.path.exists(caminho_do_arquivo):
    print(f"ERRO: Arquivo não encontrado em: {caminho_do_arquivo}")
    print("Verifique se o caminho está correto e se o Google Drive está montado.")
else:
    print("Arquivo encontrado! Lendo...")

    # Carrega o experimento
    exp = po.MSExperiment()
    po.MzMLFile().load(caminho_do_arquivo, exp)

    # Verifica quantos cromatogramas estão no arquivo
    chroms = exp.getChromatograms()
    print(f"\nO arquivo contém {len(chroms)} cromatogramas (transições).")

    print("\n--- Lista de Cromatogramas (Transições) Encontrados ---")
    for i, chrom in enumerate(chroms):
        # O 'getNativeID()' geralmente nos diz o que é a transição
        print(f"  ID {i}: {chrom.getNativeID()} | Número de pontos: {len(chrom.get_peaks()[0])}")

Arquivo encontrado! Lendo...

O arquivo contém 6 cromatogramas (transições).

--- Lista de Cromatogramas (Transições) Encontrados ---
  ID 0: TIC | Número de pontos: 1174
  ID 1: SRM SIC 363.035,121.19 | Número de pontos: 587
  ID 2: SRM SIC 363.035,327.13 | Número de pontos: 587
  ID 3: SRM SIC 366.1,124.09 | Número de pontos: 587
  ID 4: SRM SIC 366.1,330.1 | Número de pontos: 587
  ID 5: Pump Pressure 1 | Número de pontos: 480


In [ ]:
!pip install pyopenms openpyxl -q

import os
import pyopenms as po
import numpy as np
import pandas as pd
from tqdm import tqdm

path_mzml_folder = '/content/drive/MyDrive/Trabalho_RNAP/dados_mzML/'

path_master_sheet = '/content/drive/MyDrive/Trabalho_RNAP/labels.xlsx'

path_output_folder = '/content/drive/MyDrive/Trabalho_RNAP/dados_finais_cv/'

indices_para_extrair = [1, 2, 3, 4]
comprimento_alvo = 587


print("Iniciando POP 1.1 (CV): Preparação de Dados para Cross-Validation...")

os.makedirs(path_output_folder, exist_ok=True)

try:
    print(f"Carregando planilha-mestra: {path_master_sheet}")
    df_mestres = pd.read_excel(path_master_sheet, dtype={
        'NomeArquivo': str,
        'AmostraMae': str
    })


    df_mestres['RazaoArea'] = df_mestres['RazaoArea'].astype(str).str.replace(',', '.').astype(float)
    df_mestres['ConcMedia'] = df_mestres['ConcMedia'].astype(str).str.replace(',', '.').astype(float)

    label_map = {}
    for index, row in df_mestres.iterrows():
        label_map[row['NomeArquivo']] = (
            row['AmostraMae'],
            row['RazaoArea'],
            row['ConcMedia']
        )
    print(f"Encontrados {len(label_map)} rótulos únicos na planilha-mestra.")

except FileNotFoundError:
    print(f"ERRO CRÍTICO: Planilha-mestra não encontrada em {path_master_sheet}")
    raise
except KeyError as e:
    print(f"ERRO CRÍTICO: Coluna não encontrada na planilha: {e}")
    print("Verifique os nomes dos cabeçalhos (NomeArquivo, AmostraMae, RazaoArea, ConcMedia).")
    raise

print(f"Iniciando varredura e processamento dos arquivos em {path_mzml_folder}...")

lista_X_sinal = []
lista_Y_razao = []
lista_Y_conc = []
lista_grupos_mae = []

todos_os_arquivos_mzml = [f for f in os.listdir(path_mzml_folder) if f.endswith('.mzML')]

for nome_arquivo in tqdm(todos_os_arquivos_mzml, desc="Processando arquivos .mzML"):

    if nome_arquivo in label_map:
        amostra_mae, razao_area, conc_ind = label_map[nome_arquivo]
        caminho_completo = os.path.join(path_mzml_folder, nome_arquivo)
        matriz_da_amostra = np.zeros((comprimento_alvo, len(indices_para_extrair)))
        dados_encontrados = True

        try:
            exp = po.MSExperiment()
            po.MzMLFile().load(caminho_completo, exp)
            chroms = exp.getChromatograms()

            for i, idx in enumerate(indices_para_extrair):
                chrom = chroms[idx]
                intensidades = chrom.get_peaks()[1]
                tamanho_atual = len(intensidades)

                if tamanho_atual <= comprimento_alvo:
                    matriz_da_amostra[0:tamanho_atual, i] = intensidades
                else:
                    matriz_da_amostra[:, i] = intensidades[0:comprimento_alvo]

        except Exception as e:
            print(f"\nAviso: Erro ao processar o arquivo {nome_arquivo}: {e}. Pulando.")
            dados_encontrados = False

        if dados_encontrados:
            lista_X_sinal.append(matriz_da_amostra)
            lista_Y_razao.append(razao_area)
            lista_Y_conc.append(conc_ind)
            lista_grupos_mae.append(amostra_mae)

    else:
        pass

print(f"\nProcessamento concluído. {len(lista_X_sinal)} amostras válidas foram processadas e alinhadas.")

X_all = np.stack(lista_X_sinal, axis=0)
y_razao_all = np.array(lista_Y_razao)
y_conc_all = np.array(lista_Y_conc)
groups_all = np.array(lista_grupos_mae)

print(f"Salvando 4 arquivos de dados finais em: {path_output_folder}")

np.save(os.path.join(path_output_folder, 'X_all.npy'), X_all)
np.save(os.path.join(path_output_folder, 'y_razao_all.npy'), y_razao_all)
np.save(os.path.join(path_output_folder, 'y_conc_all.npy'), y_conc_all)
np.save(os.path.join(path_output_folder, 'groups_all.npy'), groups_all)

print("\n--- SUCESSO! PREPARAÇÃO DE DADOS PARA CV CONCLUÍDA! ---")
print(f"Shapes Finais: X={X_all.shape}, y_razao={y_razao_all.shape}, y_conc={y_conc_all.shape}, groups={groups_all.shape}")

Iniciando POP 1.1 (CV): Preparação de Dados para Cross-Validation...
Carregando planilha-mestra: /content/drive/MyDrive/Trabalho_RNAP/labels.xlsx
Encontrados 116 rótulos únicos na planilha-mestra.
Iniciando varredura e processamento dos arquivos em /content/drive/MyDrive/Trabalho_RNAP/dados_mzML/...


Processando arquivos .mzML: 100%|██████████| 115/115 [00:02<00:00, 52.77it/s]


Processamento concluído. 113 amostras válidas foram processadas e alinhadas.
Salvando 4 arquivos de dados finais em: /content/drive/MyDrive/Trabalho_RNAP/dados_finais_cv/

--- SUCESSO! PREPARAÇÃO DE DADOS PARA CV CONCLUÍDA! ---
Shapes Finais: X=(113, 587, 4), y_razao=(113,), y_conc=(113,), groups=(113,)
